In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U git+https://github.com/huggingface/transformers.git
!pip3 install -q -U git+https://github.com/huggingface/peft.git
!pip3 install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install -q -U einops
!pip3 install -q -U safetensors
!pip3 install -q -U torch
!pip3 install -q -U xformers

In [1]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [2]:
# My version with smaller chunks on safetensors for low RAM environments
model_id = "meta-llama/Llama-2-7b-chat-hf"

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2023-08-14 21:41:26.927246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 21:41:28.010468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto",
        quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)